In [0]:
""" Input is a coloured 300x300 image """
#    (Balanced Data)

from google.colab import drive
from random import shuffle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Input, ZeroPadding2D, ZeroPadding1D, Dropout, BatchNormalization, GaussianNoise
from tensorflow.keras import metrics
from tensorflow.keras.metrics import AUC
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [0]:
drive.mount('/content/drive')

# Loading data
x_data = np.load('/content/drive/My Drive/xtrain.npy')
y_data = np.load('/content/drive/My Drive/ytrain.npy')

print(x_data.shape)
print(y_data.shape)

In [0]:
 # Shuffling the data
ind_list = [i for i in range(298)]
shuffle(ind_list)


# Shuffled data
x_data_new = x_data[ind_list,:,:,:]
y_data_new = y_data[ind_list,]

In [0]:
# Architecture of the Network

inputs = Input(shape=x_data_new.shape[1:])

x = GaussianNoise(0.1)(inputs)

x = Conv2D(filters=16, kernel_size=(3,3), strides=1, padding='valid', activation='relu', input_shape=x_data_new.shape[1:], kernel_regularizer=regularizers.l1_l2(0.01))(x)
x = MaxPool2D(pool_size=(3,3), strides=1)(x)
x = BatchNormalization()(x)


x = Conv2D(filters=9, kernel_size=(4,4), strides=1, padding='valid', activation='relu', input_shape=x_data_new.shape[1:], kernel_regularizer=regularizers.l1_l2(0.01))(x)
x = MaxPool2D(pool_size=(4,4), strides=1)(x)
x = BatchNormalization()(x)

x = Dropout(rate=0.50)(x)

x = Flatten()(x)

x = Dense(140, activation='relu', kernel_regularizer=regularizers.l1_l2(0.03))(x)

x = Dropout(rate=0.50)(x)

x = BatchNormalization()(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=predictions)

opt = Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[tf.keras.metrics.AUC()])

datagen = ImageDataGenerator(
  rotation_range=10,
  zoom_range=0.1,
  width_shift_range=0.1,
  height_shift_range=0.1,
  validation_split = 0.2,
  featurewise_std_normalization = True,
  featurewise_center = True
)
# This also includes normalisation


batch_size = 32
epochs = 35

datagen.fit(x_data_new)

# Separating training and validation data
training_generator = datagen.flow(x_data_new, y_data_new, batch_size = batch_size,subset='training',seed=7) 
validation_generator = datagen.flow(x_data_new, y_data_new, batch_size= batch_size,subset='validation',seed=7)


history = model.fit(training_generator, validation_data = validation_generator,
                    steps_per_epoch = (len(x_data_new)*0.8) // batch_size, epochs = epochs, validation_steps=(len(x_data_new)*0.2)//batch_size,verbose=1, callbacks=None, shuffle = True)



In [0]:
# Plotting the confusion matrix
true = validation_generator
truelabel = true[0][1].tolist() + true[1][1].tolist()
trueimg = true[0][0].tolist() + true[1][0].tolist()

predict = model.predict(trueimg)

predict = [np.round(x) for x in predict]
cm=confusion_matrix(truelabel,predict)
cm

In [0]:
"""Visualizing"""

#Training/Testing accuracy over epochs
plt.plot(history.history['auc'], label='training accuracy')
plt.plot(history.history['val_auc'], label='testing accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

#Loss Curve
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='testing loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()
